In [1]:
from numpy import *
from PIL import *
import pickle
from pylab import *
import os

In [2]:
import sift
import dsift
dsift = reload(dsift)
import imtools
imtools = reload(imtools)

In [3]:
def read_gesture_features_labels(path):
    # make a list of the files with .dsift at the end
    featlist = [os.path.join(path, f) for f in os.listdir(path)
               if f.endswith('.dsift')]
    
    # read features
    features = []
    for featfile in featlist:
        l, d = sift.read_features_from_file(featfile)
        features.append(d.flatten())
    features = array(features)
    
    # generate labels
    labels = [featfile.split('/')[-1][0] for featfile in featlist]
    
    return features, array(labels)

In [138]:
features, labels = read_gesture_features_labels('train/')
test_features, test_labels = read_gesture_features_labels('test/')
classnames = unique(labels)

In [140]:
def convert_labels(labels, transl):
    return [transl[l] for l in labels]

In [141]:
features = map(list, features)
test_features = map(list, test_features)

In [142]:
# Alphabetic label is translated to numbers, and vice versa
transl = {}
for i, c in enumerate(classnames):
    transl[c], transl[i] = i, c

In [143]:
from svmutil import *

In [144]:
prob = svm_problem(convert_labels(labels, transl), features)   # Labels 'A', 'B', ... is converted to numbers
param = svm_parameter('-t 0')   # use linear kernel

In [145]:
model = svm_train(prob, param)

In [146]:
res = svm_predict(convert_labels(labels, transl), features, model)

Accuracy = 100% (174/174) (classification)


In [147]:
res = svm_predict(convert_labels(test_labels, transl), test_features, model)[0]
res = convert_labels(res, transl)

Accuracy = 80.7692% (168/208) (classification)


In [148]:
acc = sum(1.0*(res==test_labels)) / len(test_labels)
print 'Accuracy:', acc

Accuracy: 0.807692307692


In [149]:
def print_confusion(res, test_labels, classnames):
    
    n = len(classnames)
    
    class_ind = dict([(classnames[i], i) for i in range(n)])
    
    confuse = zeros((n, n))
    for i in range(len(test_labels)):
        confuse[class_ind[res[i]], class_ind[test_labels[i]]] += 1
    
    print 'Confusion matrix for'
    print classnames
    print confuse

In [150]:
print_confusion(res, test_labels, classnames)

Confusion matrix for
['A' 'B' 'C' 'F' 'P' 'V']
[[ 26.   0.   4.   4.   2.   0.]
 [  0.  28.   0.  13.   0.   0.]
 [  0.   0.  30.   0.   0.   0.]
 [  0.   1.   0.  27.   0.   0.]
 [  0.   2.   0.   0.  30.   1.]
 [  3.   1.   2.   3.   4.  27.]]


In [151]:
import pca

In [196]:
V, S, m = pca.pca(array(features))
V = V[:100]

In [197]:
features2 = [list(dot(V, array(f)-m)) for f in features]

In [198]:
prob2 = svm_problem(convert_labels(labels, transl), features2)   # Labels 'A', 'B', ... is converted to numbers
param2 = svm_parameter('-t 0')   # use linear kernel
model2 = svm_train(prob2, param2)

In [199]:
res2 = svm_predict(convert_labels(labels, transl), features2, model2)[0]
res2 = convert_labels(res2, transl)

Accuracy = 100% (174/174) (classification)


In [200]:
acc2 = sum(1.0*(res2==labels)) / len(labels)
print 'Accuracy:', acc2

Accuracy: 1.0


In [201]:
test_features2 = [dot(V, array(f)-m) for f in test_features]
test_features2 = map(list, test_features2)

In [202]:
res3 = svm_predict(convert_labels(test_labels, transl), test_features2, model2)[0]
res3 = convert_labels(res3, transl)
acc3 = sum(1.0*(res3==test_labels)) / len(test_labels)
print 'Accuracy:', acc3

Accuracy = 79.8077% (166/208) (classification)
Accuracy: 0.798076923077
